# Runs brainvisa preprocessing

This notebook creates the whole brainvisa preprocessing pipeline to feed to deep learning algorithms.
Note that you need brainvisa installed or you need to run the server under the brainvisa singularity.

You also need to have access to the HCP database (the HCP folder lies as subfolder of root_unsupervised) and a path to the root of the supervised folder (/neurospin for persons present at neurospin)

# Sets root directories

Contrary to the other notebooks, this one relies on data that are outside the deep_folding/data folder.

In [12]:
# This is the root of the HCP directory
# Could be either /tgcc, or /nfs/tgcc, for example
root_unsupervised = '/tgcc'  

# This is the root of the supervised directory
# Could be either /neuropsin, or /nfs/neurospin, for example
root_supervised = '/neurospin' 

# Imports

General imports

In [3]:
import sys
import os
from os.path import join
import glob
import json
import inspect

Deep_folding imports

In [4]:
from deep_folding.anatomist_tools import crop_definition
from deep_folding.anatomist_tools import dataset_gen_pipe
print(inspect.getfile(crop_definition))
print(inspect.getfile(dataset_gen_pipe))

/neurospin/dico/lguillon/deep_folding/deep_folding/anatomist_tools/crop_definition.py
/neurospin/dico/lguillon/deep_folding/deep_folding/anatomist_tools/dataset_gen_pipe.py


Constants

In [5]:
_ALL_SUBJECTS = -1

In [6]:
out_voxel_size = 1

In [7]:
number_subjects_supervised = 10 # Number of subjects for which we determine the box. We can set it to _ALL_SUBJECTS

In [8]:
number_subjects = 10 # Number of subjects for which we generate the crops. We can set it to _ALL_SUBJECTS

# Creates useful functions

In [9]:
def check_directory(directory_path):
    """Checks directory path and returns absolute path"""
    directory_path = os.path.abspath(directory_path)
    if os.path.isdir(directory_path):
        print((directory_path + ' is a directory'))
    else:
        print((directory_path + ' does not exist or is not a directory.'))
    return directory_path

# Variables used by all sub-computations

The following boolean variables decide which pprocessing to run:

In [10]:
run_bbox = True  # If set to True, it generates new bounding boxes
run_crop = True  # If set to True, it generates crops

We now assign path names and other user-specific variables.

The unsupervised source directory is where the unsupervised database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [13]:
unsupervised_src_dir = check_directory(join(root_unsupervised, 'hcp'))

/mnt/n4hhcp/hcp is a directory


The supervised source directories are where lies the database that has been manually labelled. It is a list of full pathes towards the manually labelled datasets.

In [14]:
human_supervised_dir = join(root_supervised, 'dico/data/bv_databases/human')
supervised_src_dir = [check_directory(join(human_supervised_dir, 'pclean/all'))
                     ]
path_to_graph = ["t1mri/t1/default_analysis/folds/3.3/base2018_manual"
                 ]

/neurospin/dico/data/bv_databases/human/pclean/all is a directory


# Generates bounding boxes

### User variables

In [15]:
bbox_dir = check_directory(join(root_supervised, 'dico/data/deep_folding/test', 'bbox'))

/neurospin/dico/data/deep_folding/test/bbox is a directory


In [23]:
mask_dir = check_directory(join(root_supervised, 'dico/data/deep_folding/current/mask/2mm'))

/neurospin/dico/data/deep_folding/current/mask/2mm is a directory


Lists the sulci of the left side that we want to analyze:

In [16]:
sulci_left = ['S.T.s.ter.asc.ant.', 'S.T.s.ter.asc.post.']

Lists the sulci of the right side that we want to analyze:

In [17]:
sulci_right = ['S.T.s.ter.asc.ant.', 'S.T.s.ter.asc.post.']

### Generates bounding boxes (actual program)

We first call crop_definition help as if called from a command line:

In [18]:
args = "--help"
argv = args.split(' ')
crop_definition.main(argv)

usage: crop_definition.py [-h] [-s SRC_DIR [SRC_DIR ...]] [-t BBOX_DIR]
                          [-m MASK_DIR] [-u SULCUS] [-i SIDE]
                          [-p PATH_TO_GRAPH] [-n NB_SUBJECTS]
                          [-v OUT_VOXEL_SIZE]

Computes mask and bounding box around the named sulcus

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR [SRC_DIR ...], --src_dir SRC_DIR [SRC_DIR ...]
                        Source directory where the MRI data lies. If there are
                        several directories, add all directories one after the
                        other. Example: -s DIR_1 DIR_2. Default is :
                        /neurospin/dico/data/bv_databases/human/pclean/all
  -t BBOX_DIR, --bbox_dir BBOX_DIR
                        Target directory where to store the output bbox json
                        files. Default is :
                        /neurospin/dico/data/deep_folding/test/bbox
  -m MASK_DIR, --mask_dir MASK_DIR
    

In [19]:
print(sulci_left, sulci_right)

['S.T.s.ter.asc.ant.', 'S.T.s.ter.asc.post.'] ['S.T.s.ter.asc.ant.', 'S.T.s.ter.asc.post.']


We now run the actial program.
This saves as json files in bbox_dir the bounding box characteristics:

In [20]:
if run_bbox:
    for sulcus in sulci_left:
        crop_definition.bounding_box(src_dir=supervised_src_dir, 
                                  path_to_graph=path_to_graph,
                                  bbox_dir=bbox_dir,
                                  sulcus=sulcus,
                                  side='L',
                                  number_subjects=number_subjects_supervised,
                                  out_voxel_size=out_voxel_size)
    for sulcus in sulci_right:
        crop_definition.bounding_box(src_dir=supervised_src_dir, 
                                  path_to_graph=path_to_graph,
                                  bbox_dir=bbox_dir,
                                  sulcus=sulcus,
                                  side='R',
                                  number_subjects=number_subjects_supervised,
                                  out_voxel_size=out_voxel_size)

{'subject': 'sujet01', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'sujet01/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Lsujet01*.arg'}
{'subject': 's12590', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 's12590/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Ls12590*.arg'}
{'subject': 'ammon', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'ammon/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Lammon*.arg'}
{'subject': 'vayu', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'vayu/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Lvayu*.arg'}
{'subject': 'osiris', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'osiris/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Losiris*.arg'}
{'subject': 's12919', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/huma

box (MNI 152) min: [136.58125347 156.5948479   85.3791845 ]
box (MNI 152) max: [156.86153672 179.19052237  95.47051659]
{'subject': 'osiris', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'osiris/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Losiris*.arg'}
box (MNI 152) min: [125.3764355  156.56988032  71.37490211]
box (MNI 152) max: [150.40756988 180.00372698 104.96537452]
{'subject': 's12919', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 's12919/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Ls12919*.arg'}
box (MNI 152) min: [131.13273095 157.90875336  82.57131003]
box (MNI 152) max: [150.20448099 181.92393347  92.84743663]
{'subject': 'sujet02', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'sujet02/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Lsujet02*.arg'}
box (MNI 152) min: [131.60826326 156.1889721   86.38205793]
box (MNI 152) max: [15

{'subject': 's12919', 'side': 'R', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 's12919/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Rs12919*.arg'}
{'subject': 'sujet02', 'side': 'R', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'sujet02/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Rsujet02*.arg'}
{'subject': 'sujet07', 'side': 'R', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'sujet07/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Rsujet07*.arg'}
{'subject': 'cronos', 'side': 'R', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'cronos/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Rcronos*.arg'}
{'subject': 'beflo', 'side': 'R', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'beflo/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Rbeflo*.arg'}
/neurospin/dico/data/deep_folding/test/mask/R/S.T.s.ter.asc.post._right.

# Generates crops

### User variables 

In [18]:
interp = 'nearest'

In [24]:
crop_dir = check_directory(join(root_supervised, 'dico/data/deep_folding/test', 'crops'))

/neurospin/dico/data/deep_folding/test/crops is a directory


### Generates crops (actual program)

We now save in {crop_dir}/Lcrops and {crop_dir}/Rcrops the actial crops based on bounding boxes:

In [29]:
if run_crop:
    # Runs on left hemisphere
    dataset_gen_pipe.dataset_gen_pipe(src_dir=unsupervised_src_dir,
                                      tgt_dir=crop_dir,
                                      bbox_dir=bbox_dir,
                                      cropping='bbox',
                                      list_sulci=sulci_left,
                                      side='L',
                                      number_subjects=number_subjects)
    # Runs on right hemisphere
    dataset_gen_pipe.dataset_gen_pipe(src_dir=unsupervised_src_dir,
                                      tgt_dir=crop_dir,
                                      bbox_dir=bbox_dir,
                                      cropping='bbox',
                                      list_sulci=sulci_right,
                                      side='R',
                                      number_subjects=number_subjects)
    print("Done")

list_subjects =  ['585862', '586460', '727654', '210617', '123420', '145127', '156536', '887373', '810843', '896778']


SUBMITTING | :   0%|          | 0/10 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/10 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/10 [00:00<?, ?it/s]

list_subjects =  ['585862', '586460', '727654', '210617', '123420', '145127', '156536', '887373', '810843', '896778']


SUBMITTING | :   0%|          | 0/10 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/10 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/10 [00:00<?, ?it/s]

Done


We now sort generated files (we do it in date order; indeed, we may find older files in folder as we don't suppress files before writing new ones):

In [30]:
files = glob.glob(f"{crop_dir}/Lcrops/*.nii.gz")
files.sort(key=os.path.getmtime, reverse=True)
print("\n".join(files[:number_subjects]))

/neurospin/dico/data/deep_folding/test/crops/Lcrops/887373_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/210617_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/145127_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/810843_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/727654_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/585862_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/123420_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/896778_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/586460_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Lcrops/156536_normalized.nii.gz


In [31]:
files = glob.glob(f"{crop_dir}/Rcrops/*.nii.gz")
files.sort(key=os.path.getmtime, reverse=True)
print("\n".join(files[:number_subjects]))

/neurospin/dico/data/deep_folding/test/crops/Rcrops/145127_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/810843_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/887373_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/896778_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/586460_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/210617_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/727654_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/585862_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/123420_normalized.nii.gz
/neurospin/dico/data/deep_folding/test/crops/Rcrops/156536_normalized.nii.gz


### Generates mask-based crops

In [36]:
sulci_left = ['paracingular.', 'F.C.M.ant.']
sulci_right = ['S.T.s.ter.asc.ant.', 'S.T.s.ter.asc.post.']

In [39]:
if run_crop:
    # Runs on left hemisphere
    dataset_gen_pipe.dataset_gen_pipe(src_dir=unsupervised_src_dir,
                                      tgt_dir=crop_dir,
                                      mask_dir=mask_dir,
                                      list_sulci=sulci_left,
                                      side='L',
                                      number_subjects=number_subjects)
    # Runs on right hemisphere
    dataset_gen_pipe.dataset_gen_pipe(src_dir=unsupervised_src_dir,
                                      tgt_dir=crop_dir,
                                      mask_dir=mask_dir,
                                      list_sulci=sulci_right,
                                      side='R',
                                      number_subjects=number_subjects)
    print("Done")

list_subjects =  ['585862', '586460', '727654', '210617', '123420', '145127', '156536', '887373', '810843', '896778']


SUBMITTING | :   0%|          | 0/10 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/10 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/10 [00:00<?, ?it/s]

list_subjects =  ['585862', '586460', '727654', '210617', '123420', '145127', '156536', '887373', '810843', '896778']


SUBMITTING | :   0%|          | 0/10 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/10 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/10 [00:00<?, ?it/s]

Done
